In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .master("local") \
     .appName("Word Count") \
     .getOrCreate()

In [113]:
class MatchData:
    def __init__(self, line):
        #print("line=",line)
        pieces = line['value'].split(',')
        self.id1 = int(pieces[0])
        self.id2 = int(pieces[1])
        self.scores = [float(i) if i != '?' else float('nan') for i in pieces[2:11]]
        if pieces[11] == 'TRUE':
            self.matched = True
        else:
            self.matched = False

def isHeader(line):
    if "id_1" in line: 
        return True
    else:
        return False

def parse(line):
    return MatchData(line)

In [114]:
log_text = spark.read.text("/tmp/linkage/linkage/block*csv")

In [115]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

get_isHeader_udf = udf(isHeader, BooleanType())
parse_udf = udf(parse)

In [116]:
log_text.value
noheader = log_text.filter(get_isHeader_udf(log_text['value']) == False)

In [ ]:
noheader.foreach(parse)

In [ ]:
noheader.cache()